## Notebook for use case digital posting assistant - stage1
### Module B accounting assignment
#### Ojectives
- In this module we will develop the accounting assignment regarding the given booking information from user

#### Processing steps from concept
B0 - preparation

B1 - input: get booking information business case from user

B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 

B3 - answer: create relevant accounting assigments for business case with LLM

B4 - output: give answer with accounting assigments to user

### B0 - Setup and configuration Modul B

The following setup-steps where processed:

* B0.0 Start SAP instances
* B0.1 install py-packages
* B0.2 load env-variables from config.json-file
* B0.3 setup and test connection to HANA DB
* B0.4 setup LLM-Connection to SAP AI-HUB
* B0.5 create SAP HANA-VectorStore interface with table 


### B0.0 Start SAP Instances

- Start BTP Cockpit
- Start SAP Build Dev Space
- Start HANA DB

In [ ]:
# B0.1 install py-packages
# RESET KERNEL AFTER INSTALLATION

%pip install --upgrade pip

%pip install hdbcli --break-system-packages
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install folium --break-system-packages
%pip install ipywidgets --break-system-packages
%pip install pypdf
%pip install -U ipykernel
%pip install hana-ml
%pip install langchain
%pip install hdbcli
%pip install sqlalchemy-hana

In [ ]:
# B0.2 load env-variables from config.json-file
# This script loads environment variables from a JSON configuration file
# and sets them in the current environment. It raises an error if the file does not exist
# or if the JSON file is malformed.

import json
import os


def load_env_variables(config_file):
    """
    Load environment variables from a JSON configuration file.

    Args:
        config_file (str): Path to the JSON configuration file.

    Returns:
        dict: A dictionary containing the environment variables.
    """
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"The configuration file {config_file} does not exist.")
    
    try:
        with open(config_file, 'r') as file:
            env_variables = json.load(file)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON from the configuration file {config_file}: {e}")
    
    for key, value in env_variables.items():
        # Convert non-string values to strings before setting them in os.environ
        if isinstance(value, dict):
            value = json.dumps(value)  # Convert dictionaries to JSON strings
        os.environ[key] = str(value)
    
    return env_variables

# Example usage
config_file = "/home/user/.aicore/config.json"
try:
    env_variables = load_env_variables(config_file)
    print(f"Loaded environment variables: {env_variables}")
except (FileNotFoundError, ValueError) as e:
    print(e)

In [ ]:
# B0.2 Test connection with env-Variables to SAP AI core

from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="SAP Generative AI Hub is awesome!",
    model_name="text-embedding-ada-002"
    
)
print(response.data)

In [ ]:
# B0.3 Setup and test connection to HANA DB

import os
# from hana_ml import ConnectionContext
from hdbcli import dbapi

# Fetch environment variables
hdb_host_address = os.getenv("hdb_host_address")
hdb_user = os.getenv("hdb_user")
hdb_password = os.getenv("hdb_password")
hdb_port = os.getenv("hdb_port")

# Debugging: Print non-sensitive environment variables
print(f"hdb_host_address: {hdb_host_address}")
print(f"hdb_user: {hdb_user}")
print(f"hdb_port: {hdb_port}")

# Ensure variables are defined
if not all([hdb_host_address, hdb_user, hdb_password, hdb_port]):
    raise ValueError("One or more HANA DB connection parameters are missing.")

# Convert port to integer
hdb_port = int(hdb_port)

# Create a connection to the HANA database
# hana_connection = ConnectionContext(
#     address=hdb_host_address,
#     port=hdb_port,
#     user=hdb_user,
#     password=hdb_password,
#     encrypt=True
# )

# Test the connection
# print("HANA DB Version:", hana_connection.hana_version())
# print("Current Schema:", hana_connection.get_current_schema())

hana_connection = dbapi.connect(
    address=hdb_host_address,
    port=hdb_port,
    user=hdb_user,
    password=hdb_password,
    #encrypt=True
    autocommit=True,
    sslValidateCertificate=False,
)





In [ ]:
#B0.4 Setup LLM-Connection to SAP AI-HUB

import os
import dotenv
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAI

# Lade aicore_model_name aus der Umgebungskonfiguration
aicore_model_name = str(os.getenv("AICORE_DEPLOYMENT_MODEL"))

# Überprüfe, ob die Variable definiert ist
if not aicore_model_name:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")

llm = ChatOpenAI(proxy_model_name=aicore_model_name)
#llm = OpenAI(proxy_model_name=aicore_model_name)

if not llm:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")
else:
    print(f"""Parameter LLM-Model-Name: {aicore_model_name} wurde erfolgreich geladen.""")


In [7]:
# B0.5 - create SAP HANA-VectorStore interface with table 
# check table content with sap-hana-database explorer: select * from ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN

from langchain_community.vectorstores.hanavector import HanaDB

vector_table_name = str(os.getenv('hdb_table_name')) + "_" + str(os.getenv('hdb_user')) 

hana_database = HanaDB(
    embedding = embeddings, 
    connection = hana_connection, 
    table_name = vector_table_name
)

try:
    print(f"""
    Successfully created SAP HANA VectorStore interface: {hana_database.connection}
    and SAP HANA table: {vector_table_name}.
    """)
except Exception as e:
    print(e)



    Successfully created SAP HANA VectorStore interface: <dbapi.Connection Connection object : bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com,443,DBADMIN,DB@hmx04,True>
    and SAP HANA table: ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN.
    


In [ ]:
# check B0.5 - query to the SAP HANA table to verify embeddings

cursor = hana_connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{hana_database.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

print(vectors[:1])

# for vector in vectors:
#     print(vector)

### processing functions Modul B

* B1 - input: get booking information business case from user

* B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 

* B3 - answer: create relevant accounting assigments for business case with LLM

* B4 - output: give answer with accounting assigments to user

In [9]:
# 1 - input: get booking information business case from user
# the function uses the store input from user

import ipywidgets as widgets
from IPython.display import display, clear_output

# Chat Interface erstellen
input_text = widgets.Text(
    value='',
    placeholder='Geben Sie Ihre Buchungsinformationen ein...',
    description='Eingabe:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

output_text = widgets.Output(
    layout=widgets.Layout(width='80%', border='1px solid black', padding='10px')
)

def on_submit(b):
    with output_text:
        clear_output()
        print(f"Ihre Eingabe: {input_text.value}")
        # Hier können später die Verarbeitungsfunktionen aufgerufen werden
        input_text.value = ''  # Eingabefeld leeren

submit_button = widgets.Button(
    description='Senden',
    button_style='primary',
    tooltip='Klicken Sie hier, um die Eingabe zu senden'
)

submit_button.on_click(on_submit)

# Layout erstellen
vbox = widgets.VBox([
    widgets.HTML('<h3>Buchungsinformationen Chat</h3>'),
    input_text,
    submit_button,
    output_text
])

display(vbox)


In [8]:
# function A4.3: Check retrievaleval for the embeddings in the SAP-Hana-Database

import os
from langchain import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = prompt | llm | parser

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

print(llm_chain.invoke({'question': question}))
answer = llm_chain.invoke({'question': question})
print(answer)


NameError: name 'RunnablePassthrough' is not defined

In [ ]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="map_reduce"

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

map_template = """
Analysiere den folgenden Kontext und extrahiere relevante Informationen zur Kontierung:

{context}

Frage: {question}

Gib die relevanten Informationen in einem kurzen Zwischenergebnis zurück.

"""

reduce_template = """
Basierend auf den folgenden Zwischenergebnissen, erstelle eine finale Antwort:

{summerization}

Frage: {question}

Formatiere die Ergebnisse in einer Liste von JSON-Elementen mit den folgenden Schlüsseln:
"Geschäftsfall"
"Konto Soll"
"Konto Haben"

Die Ergebnisse dürfen keine json markdown codeblock syntax enthalten.
Wenn keine relevanten Informationen gefunden wurden, gib an dass Du keine Antwort kennst.
"""

MAP_PROMPT = PromptTemplate(template=map_template, input_variables=["context", "question"])
REDUCE_PROMPT = PromptTemplate(template=reduce_template, input_variables=["summerization", "question"])

chain_type_kwargs = {
    "question_prompt": MAP_PROMPT,
    "reduce_prompt": REDUCE_PROMPT
}

question = "Finde Kontierung für die Umbuchung von langfristigen Forderungen"

retriever = hana_database.as_retriever(search_kwargs={'k':20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type='map_reduce',
     verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

In [ ]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="stuff"
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """Verwende den folgenden Kontext, um die Frage am Ende zu beantworten. Wenn du die Antwort nicht kennst,
    sage einfach, dass du es nicht weißt. Versuche nicht, eine Antwort zu erfinden. Formatiere die Ergebnisse als Liste von JSON-Elementen mit den folgenden Schlüsseln:

    "Geschäftsfall",
    "Konto Soll",
    "Konto Haben"

    Füge keine JSON-Markdown-Codeblock-Syntax in die Ergebnisse ein.

    {context}

    Frage: {question}
"""

PROMPT = PromptTemplate(template=prompt_template, 
                       input_variables=["context", "question"]
                      )
    
chain_type_kwargs = {"prompt": PROMPT}

question = "Finde Kontierung für die Buchung von Rückstellungen"

count_retrieved_documents = 10

retriever = hana_database.as_retriever(search_kwargs={'k': count_retrieved_documents})
# hint: k smaller than 20 -> to much tokens

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

In [ ]:
# check function 4.2 : check retrieval from SAP HANA DB with prompt using chain_type="map_reduce" (prompt-finetuning)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt für die Map-Phase
map_prompt_template = """Analysiere den folgenden Kontext und extrahiere relevante Kontierungsinformationen für den Geschäftsfall.
Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".

{context}

Frage: {question}
"""

# Prompt für die Combine/Reduce-Phase
combine_prompt_template = """
- Fasse die folgenden Kontierungsinformationen zusammen und entferne Duplikate.
- Gib nur die relevantesten und eindeutigsten Kontierungen zurück. 
- Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".
- Wenn relevante Informationen gefunden wurden gib diese Informationen im folgenden Struktur aus:
    ## Geschäftsfall: <Bezeichnung des Geschäftsfalls>
    ## Kontierung
    Konto-Soll: <Konto-Soll> - <Bezeichnung Konto-Soll> AN Konto-Haben: <Konto-Haben> - <Bezeichnung Konto-Haben>

Beispiel:

-----------------------------
## Geschäftsfall: Bildung von Rückstellungen
    ## Kontierung
    Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L3909101 - LC Sonstige Rückstellungen
-----------------------------

{summaries}

Frage: {question}
"""

# Korrekte Struktur für chain_type_kwargs
chain_type_kwargs = {
    "question_prompt": PromptTemplate(
        template=map_prompt_template,
        input_variables=["context", "question"]
    ),
    "combine_prompt": PromptTemplate(
        template=combine_prompt_template,
        input_variables=["summaries", "question"]
    )
}

question = "Finde Kontierung für die Buchung von Bildung von Rückstellungen"

retriever = hana_database.as_retriever(search_kwargs={'k': 20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)